In [52]:
import BaseHTTPServer
import os

In [78]:
class RequestHandler(BaseHTTPServer.BaseHTTPRequestHandler):

    # Handle a GET request.
    def do_GET(self):
        try:
            
            #Figure out what exactly is being requested.
            full_path = os.getcwd()+"/cache/"+self.path.split("/")[2]+"/"+"index.html"
            
            # It doesn't exist...i.e not in the localcache
            if not os.path.exists(full_path):
                raise ServerException("'{0}' not found".format(self.path))
                
            # ...if the file exists in cache
            elif os.path.isfile(full_path):
                self.handle_file(full_path)
            
            # ...it's something we don't handle.
            else:
                raise ServerException("Unknown object '{0}'".format(self.path))
                
        # Handle errors.
        except Exception as msg:
            self.handle_error(msg)
    
    def handle_file(self, full_path):
        try:
            with open(full_path, 'rb') as reader:
                content = reader.read()
            self.send_content(content)
        except IOError as msg:
            msg = "'{0}' cannot be read: {1}".format(self.path, msg)
            self.handle_error(msg)
    
    Error_Page = """\
        <html>
        <body>
        <h1>Error accessing {path}</h1>
        <p>{msg}</p>
        </body>
        </html>
        """

    def handle_error(self, msg):
        content = self.Error_Page.format(path=self.path, msg=msg)
        self.send_content(content)
    
    # Handle unknown objects.
    def handle_error(self, msg):
        content = self.Error_Page.format(path=self.path, msg=msg)
        self.send_content(content, 404)

    # Send actual content.
    def send_content(self, content, status=200):
        self.send_response(status)
        self.send_header("Content-type", "text/html")
        self.send_header("Content-Length", str(len(content)))
        self.end_headers()
        self.wfile.write(content)

In [79]:
if __name__ == '__main__':
    serverAddress = ('', 8080)
    server = BaseHTTPServer.HTTPServer(serverAddress, RequestHandler)
    server.serve_forever()

127.0.0.1 - - [21/Sep/2017 20:20:23] "GET http://www.amazon.com/ HTTP/1.1" 200 -


KeyboardInterrupt: 

In [80]:
server.socket.close()